In [1]:
from copy import deepcopy

In [2]:
Roraima = {
    'Uiramuta': ['Pacaraima', 'Normandia'],
    'Pacaraima': ['Uiramuta', 'Normandia', 'Boa Vista', 'Amajari'],
    'Normandia': ['Uiramuta', 'Bonfim', 'Boa Vista', 'Pacaraima'],
    'Boa Vista': ['Pacaraima', 'Normandia', 'Bonfim', 'Canta', 
                  'Mucajai', 'Alto Alegre', 'Amajari'],
    'Bonfim': ['Caracarai', 'Canta', 'Boa Vista', 'Normandia'],
    'Canta': ['Bonfim', 'Caracarai', 'Iracema', 'Mucajai', 'Boa Vista'],
    'Amajari': ['Pacaraima', 'Boa Vista', 'Alto Alegre'],
    'Alto Alegre': ['Amajari', 'Boa Vista', 'Mucajai', 'Iracema'],
    'Mucajai': ['Alto Alegre', 'Boa Vista', 'Canta', 'Iracema'],
    'Iracema': ['Alto Alegre', 'Mucajai', 'Canta', 'Caracarai'],
    'Caracarai': ['Iracema', 'Canta', 'Bonfim', 'Caroebe', 'Sao Joao da Baliza', 
                  'Sao Luiz do Anaua', 'Rorainopolis'],
    'Caroebe': ['Sao Joao da Baliza', 'Caracarai'],
    'Sao Joao da Baliza': ['Rorainopolis', 'Sao Luiz do Anaua', 'Caracarai', 'Caroebe'],
    'Rorainopolis': ['Caracarai', 'Sao Luiz do Anaua', 'Sao Joao da Baliza'],
    'Sao Luiz do Anaua': ['Caracarai', 'Sao Joao da Baliza', 'Rorainopolis'],
}

class Problema:
    def __init__(self, bordas, cores):
        # bordas é um dicionário onde cada chave é uma região
        # e seu valor é uma lista de suas regiões fronteiriças
        self.bordas = bordas

        # cores é uma lista com as 4 cores a serem usadas na coloração
        self.cores = cores

class No:
    def __init__(self, mapa, regiao, cor):
        # mapa é um dicionario
        # cada chave é uma região, seu valor é sua cor atual
        # representa diferentes estados do sistema
        self.mapa = deepcopy(mapa)

        # ao criar um novo nó, passando uma região e uma cor,
        # o nó é criado com a coloração atualizada
        self.mapa[regiao] = cor

    def __str__(self):
        string = ''
        for regiao in self.mapa:
            if self.mapa[regiao] != '':
                string = string + f'{regiao}: {self.mapa[regiao]}\n'
        return string

    def __repr__(self):
        return self.__str__()
    
    def filhos(self, cores, proxima_regiao):
        filhos = []
        for cor in cores:
            filho = No(self.mapa, proxima_regiao, cor)
            filhos.append(filho)
        return filhos

class Coloracao:
    def __init__(self, problema):
        self.problema = problema
        self.fronteira = []
        self.status = 'Coloração iniciando'
        # status possíveis: 'Coloração iniciando', 'Coloração em andamento', 'Coloração finalizada'

        self.indice = 0
        # o indice guardará a posição da próxima região a ser colorida na busca

        # mapa é um dicionário com todas as regiões e suas cores
        # começa em branco
        # representa o estado inicial do sistema
        self.mapa = {}
        for regiao in self.problema.bordas:
            self.mapa[regiao] = ''

    def passo(self):
        if self.status == 'Coloração finalizada':
            print('Coloração finalizada')
            return
        
        try:
            no = self.fronteira.pop(-1)
        except IndexError:
            self.situacao = 'Coloração finalizada'
            return

        for filho in no.filhos(self.problema.cores, self.problema.bordas[self.problema.mapa]):
            self.fronteira.append(filho)
            pass

In [3]:
prob_ex = Problema(Roraima, ['branco', 'verde', 'azul', 'cinza'])
col_ex = Coloracao(prob_ex)
no_1 = No(col_ex.mapa, 'Uiramuta', 'branco')

no_1

Uiramuta: branco

In [4]:
filhos = no_1.filhos(['branco', 'verde', 'azul', 'cinza'], 'Pacaraima')

In [5]:
filhos[0]

Uiramuta: branco
Pacaraima: branco

In [6]:
filhos[1]

Uiramuta: branco
Pacaraima: verde

In [7]:
filhos[2]

Uiramuta: branco
Pacaraima: azul

In [8]:
filhos[3]

Uiramuta: branco
Pacaraima: cinza

In [12]:
netos = []
for filho in filhos:
    netos.append(filho.filhos(['branco', 'verde', 'azul', 'cinza'], 'Normandia'))

In [17]:
for i in range(4):
    for j in range(4):
        print(netos[i][j])

Uiramuta: branco
Pacaraima: branco
Normandia: branco

Uiramuta: branco
Pacaraima: branco
Normandia: verde

Uiramuta: branco
Pacaraima: branco
Normandia: azul

Uiramuta: branco
Pacaraima: branco
Normandia: cinza

Uiramuta: branco
Pacaraima: verde
Normandia: branco

Uiramuta: branco
Pacaraima: verde
Normandia: verde

Uiramuta: branco
Pacaraima: verde
Normandia: azul

Uiramuta: branco
Pacaraima: verde
Normandia: cinza

Uiramuta: branco
Pacaraima: azul
Normandia: branco

Uiramuta: branco
Pacaraima: azul
Normandia: verde

Uiramuta: branco
Pacaraima: azul
Normandia: azul

Uiramuta: branco
Pacaraima: azul
Normandia: cinza

Uiramuta: branco
Pacaraima: cinza
Normandia: branco

Uiramuta: branco
Pacaraima: cinza
Normandia: verde

Uiramuta: branco
Pacaraima: cinza
Normandia: azul

Uiramuta: branco
Pacaraima: cinza
Normandia: cinza

